In [1]:
import seaborn as sns
from tqdm import tqdm
import pandas as pd
from rpy2.robjects import r, pandas2ri
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import os

In [2]:
pandas2ri.activate()

In [3]:
from isaac.constants import BASIC_TRAINING_COLS, YOKED_TRAINING_COLS

In [4]:
def are_all_coordinates_within_bounds(trial):
    
    for coord in ["o1.x", "o2.x", "o3.x", "o4.x", "o1.y", "o2.y", "o3.y", "o4.y"]:
        if (trial[coord] < 0).any().any():
            return False
        
    for x_coord in ["o1.x", "o2.x", "o3.x", "o4.x"]:
        if (trial[x_coord] > 6).any().any():
            return False 
        
    for y_coord in ["o1.y", "o2.y", "o3.y", "o4.y"]:
        if (trial[y_coord] > 4).any().any():
            return False
    
    return True

# Experiment 1 

In [5]:
rdata_path = "data/experiment1.rdata"
r['load'](rdata_path)

'df.l','df.w','ls.clips','ls.events'


In [6]:
all_trials = r['ls.clips']

is_active = (r['df.l'].condition == '1')
responses = r['df.l'][["trueMass", "trueRelationship"]]
YOKED_PATH = "data/yoked_trials_exp1.h5"
ALTERNATIVE_TRAINING_COLS = ["object"+str(obj)+"."+attr for obj in range(1, 5) for attr in ["x", "y", "vx", "vy"]]

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [7]:
if os.path.exists(YOKED_PATH):
    os.remove(YOKED_PATH)

In [8]:
flat_trials = []
trial_i = 0

for participant in all_trials:
    for trial in participant:
        trial = pandas2ri.ri2py_dataframe(trial)
        # print(trial["idControlledObject"].nunique() > 1, is_active[trial_i])
        flat_trials.append(trial)
        trial_i += 1

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [9]:
len(flat_trials), len(is_active), len(responses)

(484, 704, 704)

In [10]:
flat_trials = np.array(flat_trials)
active_trials = flat_trials[is_active.iloc[:-220]]

In [11]:
len(active_trials)

220

In [12]:
mass_trials = []
force_trials = []

trial_counter = -1
skipped = 0

for trial in tqdm(active_trials):
    trial_counter += 1

    if set(ALTERNATIVE_TRAINING_COLS).issubset(trial.columns):
        trial = trial.rename({alt_col: col for col, alt_col in 
                              zip(BASIC_TRAINING_COLS, ALTERNATIVE_TRAINING_COLS)}, axis="columns")
    else:
        print("Not all columns are available for participant")
        skipped += 1
        continue

    if not are_all_coordinates_within_bounds(trial):
        print("Skipping trial because coordinates out of bounds")
        continue


    trial["C_none"] = (trial["idControlledObject"] == "none")
    trial["C_O1"] = (trial["idControlledObject"] == "object1")
    trial["C_O2"] = (trial["idControlledObject"] == "object2")
    trial["C_O3"] = (trial["idControlledObject"] == "object3")
    trial["C_O4"] = (trial["idControlledObject"] == "object4")

    trial = trial[YOKED_TRAINING_COLS + ["idControlledObject"]].copy()

    true_mass = responses.iloc[trial_counter]["trueMass"]
    trial["A"] = (true_mass == "A")
    trial["B"] = (true_mass == "B")
    trial["same"] = (true_mass == "same")

    true_relationship = responses.iloc[trial_counter]["trueRelationship"]
    trial["attract"] = (true_relationship == "attract")
    trial["none"] = (true_relationship == "none")
    trial["repel"] = (true_relationship == "repel")

    trial.to_hdf(path_or_buf=YOKED_PATH, key="trial_"+str(trial_counter))

100%|██████████| 220/220 [00:02<00:00, 74.63it/s]

Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for participant
Not all columns are available for 

In [13]:
skipped / 11

7.0

# Experiment 2 

In [23]:
MASS_PATH = "data/mass_trials_exp2.h5"
FORCE_PATH = "data/force_trials_exp2.h5"

rdata_path = "data/experiment2.rdata"
r['load'](rdata_path)

'df.l','df.w','ls.clips','ls.events'


In [24]:
responses = r['df.l'].query("experiment == 'experiment_4'")[["trueMass", "trueRelationship"]]
all_trials = r['ls.clips']

/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


In [25]:
if os.path.exists(MASS_PATH):
    os.remove(MASS_PATH)

if os.path.exists(FORCE_PATH):
    os.remove(FORCE_PATH)

In [26]:
trial_counter = -1

for participant in tqdm(all_trials):
    for trial in participant:
        trial_counter += 1
        trial = pandas2ri.ri2py_dataframe(trial)
        
        if trial.question.unique()[0] == "mass":
            hdf_path = MASS_PATH
        else:
            hdf_path = FORCE_PATH
        
        if not are_all_coordinates_within_bounds(trial):
            print("SKIPPING")
            if trial.question.unique()[0] == "mass":
                print("Skipping trial for mass")
            elif trial.question.unique()[0] == "relationship":
                print("Skipping trial for force")
            continue

        trial["C_none"] = (trial["idControlledObject"] == "none")
        trial["C_O1"] = (trial["idControlledObject"] == "object1")
        trial["C_O2"] = (trial["idControlledObject"] == "object2")
        trial["C_O3"] = (trial["idControlledObject"] == "object3")
        trial["C_O4"] = (trial["idControlledObject"] == "object4")
            
        trial = trial[YOKED_TRAINING_COLS + ["idControlledObject"]].copy()
                
        true_mass = responses.iloc[trial_counter]["trueMass"]
        trial["A"] = (true_mass == "A")
        trial["B"] = (true_mass == "B")
        trial["same"] = (true_mass == "same")

        true_relationship = responses.iloc[trial_counter]["trueRelationship"]
        trial["attract"] = (true_relationship == "attract")
        trial["none"] = (true_relationship == "none")
        trial["repel"] = (true_relationship == "repel")

        trial.to_hdf(path_or_buf=hdf_path, key="trial_"+str(trial_counter), format="table")

  0%|          | 0/40 [00:00<?, ?it/s]/home/hector/miniconda3/envs/diss/lib/python3.5/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)
 85%|████████▌ | 34/40 [00:32<00:06,  1.01s/it]

SKIPPING
Skipping trial for force
SKIPPING
Skipping trial for mass


100%|██████████| 40/40 [00:38<00:00,  1.01it/s]


# READING DATASET

In [18]:
from isaac.dataset import read_dataset

In [19]:
yoked_dataset = read_dataset(MASS_PATH)

100%|██████████| 400/400 [00:07<00:00, 55.93it/s]


In [20]:
yoked_dataset[0].head()

,o1.x,o1.y,o1.vx,o1.vy,o2.x,o2.y,o2.vx,o2.vy,o3.x,o3.y,...,C_O4,mouseX,mouseY,idControlledObject,A,B,same,attract,none,repel
0,2.72027,2.54902,6.82968,6.11876,1.51623,2.09434,5.78482,-4.91547,3.77683,0.93449,...,False,492.0,377.0,none,False,False,True,True,False,False
1,2.83388,2.65091,6.81662,6.11355,1.61265,2.01252,5.78554,-4.90928,3.74444,0.79531,...,False,492.0,377.0,none,False,False,True,True,False,False
2,2.94729,2.75271,6.80479,6.10797,1.70907,1.93080,5.78529,-4.90279,3.71758,0.66116,...,False,492.0,377.0,none,False,False,True,True,False,False
3,3.06053,2.85442,6.79407,6.10224,1.80548,1.84920,5.78417,-4.89625,3.69105,0.52739,...,False,492.0,377.0,none,False,False,True,True,False,False
4,3.17360,2.95602,6.78430,6.09650,1.90185,1.76770,5.78231,-4.88980,3.66849,0.50496,...,False,492.0,377.0,none,False,False,True,True,False,False


In [21]:
df_list = read_dataset("data/train_passive_trials.h5", n_trials=10)

100%|██████████| 10/10 [00:00<00:00, 174.70it/s]


In [22]:
df_list[0].head()

,o1.x,o1.y,o1.vx,o1.vy,o2.x,o2.y,o2.vx,o2.vy,o3.x,o3.y,...,o4.x,o4.y,o4.vx,o4.vy,A,B,same,attract,none,repel
0,4.555363,1.439802,-6.937327,6.954494,0.808312,0.608930,6.549034,5.213333,1.637953,0.979222,...,2.312489,1.325347,-5.571054,6.824184,False,True,False,False,True,False
1,4.440096,1.555637,-6.915997,6.950097,0.917101,0.695618,6.527328,5.201250,1.644658,0.941331,...,2.220093,1.439238,-5.543806,6.833471,False,True,False,False,True,False
2,4.325193,1.671406,-6.894182,6.946143,1.025478,0.782050,6.502610,5.185913,1.651257,0.903450,...,2.128247,1.553392,-5.510725,6.849251,False,True,False,False,True,False
3,4.210661,1.787117,-6.871926,6.942690,1.133387,0.868146,6.474587,5.165781,1.657747,0.865570,...,2.037060,1.667969,-5.471231,6.874611,False,True,False,False,True,False
4,4.096507,1.902780,-6.849294,6.939785,1.240779,0.953792,6.443504,5.138741,1.664126,0.827684,...,1.946629,1.783198,-5.425823,6.913776,False,True,False,False,True,False
